# Basic scene generation and visualization

 ## Setup code

In [ ]:
# setup vulkan
!mkdir -p /usr/share/vulkan/icd.d
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/nvidia_icd.json
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/10_nvidia.json
!mv nvidia_icd.json /usr/share/vulkan/icd.d
!mv 10_nvidia.json /usr/share/glvnd/egl_vendor.d/10_nvidia.json
!apt-get install -y --no-install-recommends libvulkan-dev
# dependencies
!pip install --upgrade mani_skill tyro

In [ ]:
!pip install torch torchvision
!pip install scene-synthesizer[recommend]
!pip install usd-core
!pip install "pyglet<2"

Clone private repo. You can specify your password or generate one-time token.

In [ ]:
# SECURE?
import os
from getpass import getpass
user = getpass('GitHub user')
pswd_or_token = getpass('GitHub password / token')
os.environ['GITHUB_AUTH'] = user + ':' + pswd_or_token
!git clone https://$GITHUB_AUTH@gitlab.2a2i.org/cv/robo/darkstore-synthesizer

Next download zip with assets from Google Drive and Robocassa assets.

In [ ]:
!gdown 1RhBw9HfoHm6uvxrFC9hxYq0JB1FRI_d-
!unzip assets.zip
!python -m mani_skill.utils.download_asset RoboCasa

In [ ]:
ASSETS_PATH = 'assets'

## Scene generation

In [ ]:
import json
import sys
import time
from tqdm import tqdm
import torch
import gymnasium as gym
import numpy as np
from IPython.display import Video

import mani_skill.envs
from mani_skill.utils.wrappers import RecordEpisode

sys.path.append('darkstore-synthesizer')
from dsynth.scene_gen.arrangements import get_assets_dict, shelf_placement
from dsynth.scene_gen.layouts.random_connectivity import add_many_products, get_orientation
from dsynth.envs.darkstore_cell_base import get_arena_data
from dsynth.envs.pick_to_cart import PickToCartEnv

In [ ]:
# Specify output directory
from pathlib import Path
output_dir = 'generated_scenes'
output_dir = Path(output_dir)
output_dir.mkdir()

In [ ]:
# Mapping from existing product assets to their model class
product_names = get_assets_dict(ASSETS_PATH)
product_names

Now darkstore floor plan is a **n x m** grid, where some cells are populated with shelves. There is example of scene specification below.



*   `room_size` is the size of grid
*   `door_coords` coordinates of special cell (darkstore entrance), which is not populated, and all shelves must be reachable from this cell
*   `n_products_on_board` - number of products per one board in shelf
*   `shelfname_to_cnt` - number of shelves on the scene per one product type. **Shelves with the same type procedurally placed next to each other.**
*   `blocked_matrix` is an array which specifies blocked cells on the layout


In [ ]:
n = 5
m = 5
scene_data = {
    "room_size": [
        n,
        m
    ],
    "door_coords": [
        0,
        0
    ],
    "n_products_on_board": 15,
    "shelfname_to_cnt": {
        "milk": 3,
        "baby": 2,
        "coke": 2
    },
    "random_shelfs": []
}
blocked_matrix = [[0] * m for _ in range(n)]

In [ ]:
is_gen, room = add_many_products(scene_data["door_coords"], blocked_matrix, scene_data["shelfname_to_cnt"])

Below we can see generated layout for 5x5 room.

In [ ]:
room

Next, generate object placements.

In [ ]:
is_rotate = get_orientation(scene_data["door_coords"], room)
scene_meta = shelf_placement(product_names, 5, scene_data["n_products_on_board"], room, is_rotate)

In [ ]:
with open(output_dir / "scene_config.json", "w") as f:
    json.dump(scene_meta, f, indent=4)

## Open generated scene in ManiSkill simulator

In [ ]:
# Get room sizes based on layout
arena_data = get_arena_data(x_cells=n, y_cells=m, height=4)

# Specify shader used for rendering
shader = 'default'

# Style ID for floor and walls from Robocasa assets
style_id = 1

# Create environment
env = gym.make('PickToCartEnv',
    robot_uids='panda_wristcam',
    scene_json = output_dir / "scene_config.json",
    assets_dir = ASSETS_PATH,
    mapping_file = 'assets/connect.json',
    style_ids = [style_id],
    num_envs=1,
    viewer_camera_configs={'shader_pack': shader},
    human_render_camera_configs={'shader_pack': shader},
    render_mode="rgb_array",
    enable_shadow=True,
    obs_mode='rgbd',
    **arena_data)

In [ ]:
# Save rollout video and trajectory
new_traj_name = time.strftime("%Y%m%d_%H%M%S")
video_path = output_dir / f"./videos_style={style_id}_shader={shader}"
env = RecordEpisode(
    env,
    output_dir=video_path,
    trajectory_name=new_traj_name,
    save_video=True,
    video_fps=30,
    avoid_overwriting_video=True
)

In [ ]:
# Run a rollout
episode_length = 100
env.reset()

zero_action = torch.zeros_like(torch.from_numpy(env.action_space.sample()))

for i in tqdm(range(episode_length)):
    obs, reward, terminated, truncated, info = env.step(zero_action)

env.close()

In [ ]:
Video(f"generated_scenes/videos_style={style_id}_shader={shader}/0.mp4", embed=True, width=640) # Watch our replay